# Snowflake Compute Cost SQL Queries — Versión para **Snowflake Notebooks**
**Fecha:** 2025-11-06
**Autor:** Adolfo Orozco
**Tags:** #snowflake #snowpark #python #sql #cost #performance #recipes

Este cuaderno está optimizado para ejecutarse dentro de **Snowflake Notebooks** usando **Snowpark**.
Cuando se ejecute fuera de Snowflake Notebooks (por ejemplo, en tu Jupyter local), hará *fallback* a una conexión estándar con credenciales.

> Consejo: ejecuta las celdas en orden. Cada sección está numerada.


## 1. Parámetros

Ajusta estos valores predeterminados para tu entorno y ventana de análisis.


In [ ]:
# 1.1 Global parameters
M_DAYS = 30
QUERY_TYPE = "COPY"
LAST_N_MINUTES = 60
MODEL_NAME = "mistral-large"
CORTEX_QUERY_ID = "query-id"
NB_NAME = "<example_nb_name>"
CP_NAME = "<example_cp_name>"

## 2. Conexión y contexto de sesión

Esta celda usa `get_active_session()` dentro de **Snowflake Notebooks**. Si ejecutas localmente, hace fallback para crear una sesión.
También establece **ROLE**, **WAREHOUSE**, **DATABASE** y **SCHEMA** para obtener resultados consistentes.


In [ ]:
# 2.1 Session and context
from __future__ import annotations
import os, getpass
import pandas as pd
try:
    from snowflake.snowpark.context import get_active_session
    from snowflake.snowpark import Session
    session = get_active_session()
    IN_SF_NB = True
except Exception:
    from snowflake.snowpark import Session
    IN_SF_NB = False

ROLE      = os.getenv("SNOWFLAKE_ROLE",      "ACCOUNTADMIN")
WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH")
DATABASE  = os.getenv("SNOWFLAKE_DATABASE",  "SNOWFLAKE")
SCHEMA    = os.getenv("SNOWFLAKE_SCHEMA",    "ACCOUNT_USAGE")

def get_session():
    global session
    if IN_SF_NB:
        return session
    conn = {
        "account":   os.getenv("SNOWFLAKE_ACCOUNT", "your_account"),
        "user":      os.getenv("SNOWFLAKE_USER", "your_user"),
        "password":  os.getenv("SNOWFLAKE_PASSWORD") or getpass.getpass("Snowflake password: "),
        "role":      ROLE, "warehouse": WAREHOUSE, "database": DATABASE, "schema": SCHEMA
    }
    session = Session.builder.configs(conn).create()
    return session

def run_sql_df(sql: str):
    s = get_session()
    return s.sql(sql).to_pandas()

_ = run_sql_df(f"""
    USE ROLE {ROLE};
    USE WAREHOUSE {WAREHOUSE};
    USE DATABASE {DATABASE};
    USE SCHEMA {SCHEMA};
""")
run_sql_df("SELECT CURRENT_ROLE() ROLE, CURRENT_WAREHOUSE() WH, CURRENT_DATABASE() DB, CURRENT_SCHEMA() SCH;")

## 3. Cómputo para warehouses
Estas consultas analizan los patrones de gasto por hora y el consumo de créditos por warehouse para encontrar picos y valores atípicos.


### 3.1 Gasto promedio por hora en Snowflake durante los últimos *m* días


In [ ]:
# 3.1.1 Hourly spend (past m days)
sql = f"""
SELECT
  start_time,
  warehouse_name,
  credits_used_compute
FROM snowflake.account_usage.warehouse_metering_history
WHERE start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
  AND warehouse_id > 0
ORDER BY 1 DESC, 2;
"""
run_sql_df(sql)

In [ ]:
# 3.1.2 Hour-of-day profile
sql = f"""
SELECT
  DATE_PART('HOUR', start_time) AS start_hour,
  warehouse_name,
  AVG(credits_used_compute)     AS credits_used_compute_avg
FROM snowflake.account_usage.warehouse_metering_history
WHERE start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
  AND warehouse_id > 0
GROUP BY 1, 2
ORDER BY 1, 2;
"""
run_sql_df(sql)

### 3.2 Consumo de créditos por warehouse


In [ ]:
# 3.2.1 Past year
sql = """
SELECT
  warehouse_name,
  SUM(credits_used_compute) AS credits_used_compute_sum
FROM snowflake.account_usage.warehouse_metering_history
WHERE start_time >= DATEADD(year, -1, CURRENT_TIMESTAMP())
GROUP BY 1
ORDER BY 2 DESC;
"""
run_sql_df(sql)

In [ ]:
# 3.2.2 Past m days
sql = f"""
SELECT
  warehouse_name,
  SUM(credits_used_compute) AS credits_used_compute_sum
FROM snowflake.account_usage.warehouse_metering_history
WHERE start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
GROUP BY 1
ORDER BY 2 DESC;
"""
run_sql_df(sql)

### 3.3 Uso del warehouse sobre el promedio de *m* días


In [ ]:
# 3.3.1 Daily vs moving average (flags >= 50% rise)
sql = f"""
WITH cte_date_wh AS (
  SELECT
    TO_DATE(start_time) AS start_date,
    warehouse_name,
    SUM(credits_used)   AS credits_used_date_wh
  FROM snowflake.account_usage.warehouse_metering_history
  GROUP BY 1, 2
)
SELECT
  start_date,
  warehouse_name,
  credits_used_date_wh,
  AVG(credits_used_date_wh) OVER (PARTITION BY warehouse_name ORDER BY start_date ROWS {M_DAYS} PRECEDING) AS credits_used_m_day_avg,
  100.0 * ((credits_used_date_wh / credits_used_m_day_avg) - 1) AS pct_over_to_m_day_average
FROM cte_date_wh
QUALIFY credits_used_date_wh > 100
   AND pct_over_to_m_day_average >= 0.5
ORDER BY pct_over_to_m_day_average DESC;
"""
run_sql_df(sql)

## 4. Cómputo para servicios en la nube


In [ ]:
# 4.1 Billed cloud services (last month)
sql = """
SELECT
  usage_date,
  credits_used_cloud_services,
  credits_adjustment_cloud_services,
  credits_used_cloud_services + credits_adjustment_cloud_services AS billed_cloud_services
FROM snowflake.account_usage.metering_daily_history
WHERE usage_date >= DATEADD(month, -1, CURRENT_TIMESTAMP())
  AND credits_used_cloud_services > 0
ORDER BY 4 DESC;
"""
run_sql_df(sql)

In [ ]:
# 4.2 CS cost by query type (last day)
sql = """
SELECT
  query_type,
  SUM(credits_used_cloud_services) AS cs_credits,
  COUNT(1)                         AS num_queries
FROM snowflake.account_usage.query_history
WHERE start_time >= TIMESTAMPADD(day, -1, CURRENT_TIMESTAMP())
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
"""
run_sql_df(sql)

In [ ]:
# 4.3 CS cost for queries of a given type (last day)
sql = f"""
SELECT *
FROM snowflake.account_usage.query_history
WHERE start_time >= TIMESTAMPADD(day, -1, CURRENT_TIMESTAMP())
  AND query_type = '{QUERY_TYPE}'
ORDER BY credits_used_cloud_services DESC
LIMIT 10;
"""
run_sql_df(sql)

In [ ]:
# 4.4 High CS ratio warehouses (last month)
sql = """
SELECT
  warehouse_name,
  SUM(credits_used)                AS credits_used,
  SUM(credits_used_cloud_services) AS credits_used_cloud_services,
  SUM(credits_used_cloud_services) / NULLIF(SUM(credits_used), 0) AS percent_cloud_services
FROM snowflake.account_usage.warehouse_metering_history
WHERE TO_DATE(start_time) >= DATEADD(month, -1, CURRENT_TIMESTAMP())
  AND credits_used_cloud_services > 0
GROUP BY 1
ORDER BY 4 DESC;
"""
run_sql_df(sql)

In [ ]:
# 4.5 Sort by time components (last N minutes)
sql = f"""
SELECT *
FROM snowflake.account_usage.query_history
WHERE start_time >= TIMESTAMPADD(minute, -{LAST_N_MINUTES}, CURRENT_TIMESTAMP())
ORDER BY
  compilation_time DESC,
  execution_time DESC,
  list_external_files_time DESC,
  queued_overload_time DESC,
  credits_used_cloud_services DESC
LIMIT 10;
"""
run_sql_df(sql)

## 5. Cómputo para Clustering Automático


In [ ]:
# 5.1 Cost history (by day, by object)
sql = """
SELECT
  TO_DATE(start_time) AS date,
  database_name,
  schema_name,
  table_name,
  SUM(credits_used)   AS credits_used
FROM snowflake.account_usage.automatic_clustering_history
WHERE start_time >= DATEADD(month, -1, CURRENT_TIMESTAMP())
GROUP BY 1, 2, 3, 4
ORDER BY 5 DESC;
"""
run_sql_df(sql)

In [ ]:
# 5.2 History & weekly average (last year)
sql = """
WITH credits_by_day AS (
  SELECT
    TO_DATE(start_time) AS date,
    SUM(credits_used)   AS credits_used
  FROM snowflake.account_usage.automatic_clustering_history
  WHERE start_time >= DATEADD(year, -1, CURRENT_TIMESTAMP())
  GROUP BY 1
)
SELECT
  DATE_TRUNC('week', date) AS week_start,
  AVG(credits_used)        AS avg_daily_credits
FROM credits_by_day
GROUP BY 1
ORDER BY 1;
"""
run_sql_df(sql)

## 6. Cómputo para Optimización de Búsqueda


In [ ]:
# 6.1 Cost history (by day, by object)
sql = """
SELECT
  TO_DATE(start_time) AS date,
  database_name,
  schema_name,
  table_name,
  SUM(credits_used)   AS credits_used
FROM snowflake.account_usage.search_optimization_history
WHERE start_time >= DATEADD(month, -1, CURRENT_TIMESTAMP())
GROUP BY 1, 2, 3, 4
ORDER BY 5 DESC;
"""
run_sql_df(sql)

In [ ]:
# 6.2 History & weekly average (last year)
sql = """
WITH credits_by_day AS (
  SELECT
    TO_DATE(start_time) AS date,
    SUM(credits_used)   AS credits_used
  FROM snowflake.account_usage.search_optimization_history
  WHERE start_time >= DATEADD(year, -1, CURRENT_TIMESTAMP())
  GROUP BY 1
)
SELECT
  DATE_TRUNC('week', date) AS week_start,
  AVG(credits_used)        AS avg_daily_credits
FROM credits_by_day
GROUP BY 1
ORDER BY 1;
"""
run_sql_df(sql)

## 7. Cómputo para Vistas Materializadas


In [ ]:
# 7.1 Cost history (by day, by object)
sql = """
SELECT
  TO_DATE(start_time) AS date,
  database_name,
  schema_name,
  table_name,
  SUM(credits_used)   AS credits_used
FROM snowflake.account_usage.materialized_view_refresh_history
WHERE start_time >= DATEADD(month, -1, CURRENT_TIMESTAMP())
GROUP BY 1, 2, 3, 4
ORDER BY 5 DESC;
"""
run_sql_df(sql)

In [ ]:
# 7.2 History & weekly average (last year)
sql = """
WITH credits_by_day AS (
  SELECT
    TO_DATE(start_time) AS date,
    SUM(credits_used)   AS credits_used
  FROM snowflake.account_usage.materialized_view_refresh_history
  WHERE start_time >= DATEADD(year, -1, CURRENT_TIMESTAMP())
  GROUP BY 1
)
SELECT
  DATE_TRUNC('week', date) AS week_start,
  AVG(credits_used)        AS avg_daily_credits
FROM credits_by_day
GROUP BY 1
ORDER BY 1;
"""
run_sql_df(sql)

## 8. Cómputo para el Servicio de Aceleración de Consultas


In [ ]:
# 8.1 QAS cost by warehouse (month-to-date)
sql = """
SELECT
  warehouse_name,
  SUM(credits_used) AS total_credits_used
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_ACCELERATION_HISTORY
WHERE start_time >= DATE_TRUNC(month, CURRENT_DATE)
GROUP BY 1
ORDER BY 2 DESC;
"""
run_sql_df(sql)

## 9. Cómputo para Snowpipe y Snowpipe Streaming


In [ ]:
# 9.1 Cumulative ingest usage (COPY vs SNOWPIPE)
sql = """
SELECT
  TO_DATE(last_load_time)                                       AS load_date,
  status,
  table_catalog_name                                            AS database_name,
  table_schema_name                                             AS schema_name,
  table_name,
  CASE WHEN pipe_name IS NULL THEN 'COPY' ELSE 'SNOWPIPE' END   AS ingest_method,
  SUM(row_count)                                                AS row_count,
  SUM(row_parsed)                                               AS rows_parsed,
  AVG(file_size)                                                AS avg_file_size_bytes,
  SUM(file_size)                                                AS total_file_size_bytes,
  SUM(file_size)/POWER(1024,1)                                  AS total_file_size_kb,
  SUM(file_size)/POWER(1024,2)                                  AS total_file_size_mb,
  SUM(file_size)/POWER(1024,3)                                  AS total_file_size_gb,
  SUM(file_size)/POWER(1024,4)                                  AS total_file_size_tb
FROM snowflake.account_usage.copy_history
GROUP BY 1,2,3,4,5,6
ORDER BY 3,4,5,1,2;
"""
run_sql_df(sql)

In [ ]:
# 9.2 Snowpipe cost history (by day, by object)
sql = """
SELECT
  TO_DATE(start_time) AS date,
  pipe_name,
  SUM(credits_used)   AS credits_used
FROM snowflake.account_usage.pipe_usage_history
WHERE start_time >= DATEADD(month, -1, CURRENT_TIMESTAMP())
GROUP BY 1, 2
ORDER BY 3 DESC;
"""
run_sql_df(sql)

In [ ]:
# 9.3 Snowpipe weekly average (last year)
sql = """
WITH credits_by_day AS (
  SELECT
    TO_DATE(start_time) AS date,
    SUM(credits_used)   AS credits_used
  FROM snowflake.account_usage.pipe_usage_history
  WHERE start_time >= DATEADD(year, -1, CURRENT_TIMESTAMP())
  GROUP BY 1
)
SELECT
  DATE_TRUNC('week', date) AS week_start,
  AVG(credits_used)        AS avg_daily_credits
FROM credits_by_day
GROUP BY 1
ORDER BY 1;
"""
run_sql_df(sql)

In [ ]:
# 9.4 Snowpipe Streaming cost (compute vs client)
sql = """
SELECT
  start_time,
  end_time,
  SUM(credits_used) AS total_credits,
  name,
  IFF(CONTAINS(name, ':'), 'streaming client cost', 'streaming compute cost') AS streaming_cost_type
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_HISTORY
WHERE service_type = 'SNOWPIPE_STREAMING'
GROUP BY ALL;
"""
run_sql_df(sql)

## 10. Cómputo para alertas y tareas sin servidor


In [ ]:
# 10.1 Alerts
sql = """
SELECT
  start_time,
  end_time,
  alert_id,
  alert_name,
  credits_used,
  schema_id,
  schema_name,
  database_id,
  database_name
FROM SNOWFLAKE.ACCOUNT_USAGE.serverless_alert_history
ORDER BY start_time, alert_id;
"""
run_sql_df(sql)

In [ ]:
# 10.2 Tasks
sql = """
SELECT
  start_time,
  end_time,
  task_id,
  task_name,
  credits_used,
  schema_id,
  schema_name,
  database_id,
  database_name
FROM snowflake.account_usage.serverless_task_history
ORDER BY start_time, task_id;
"""
run_sql_df(sql)

## 11. Cómputo para replicación


In [ ]:
# 11.1 Account replication (MTD)
sql = """
SELECT
  start_time,
  end_time,
  replication_group_name,
  credits_used,
  bytes_transferred
FROM snowflake.account_usage.replication_group_usage_history
WHERE start_time >= DATE_TRUNC('month', CURRENT_DATE());
"""
run_sql_df(sql)

In [ ]:
# 11.2 Database replication (last month)
sql = """
SELECT
  TO_DATE(start_time) AS date,
  database_name,
  SUM(credits_used)   AS credits_used
FROM snowflake.account_usage.database_replication_usage_history
WHERE start_time >= DATEADD(month, -1, CURRENT_TIMESTAMP())
GROUP BY 1, 2
ORDER BY 3 DESC;
"""
run_sql_df(sql)

In [ ]:
# 11.3 Database replication weekly average (last year)
sql = """
WITH credits_by_day AS (
  SELECT
    TO_DATE(start_time) AS date,
    SUM(credits_used)   AS credits_used
  FROM snowflake.account_usage.database_replication_usage_history
  WHERE start_time >= DATEADD(year, -1, CURRENT_TIMESTAMP())
  GROUP BY 1
)
SELECT
  DATE_TRUNC('week', date) AS week_start,
  AVG(credits_used)        AS avg_daily_credits
FROM credits_by_day
GROUP BY 1
ORDER BY 1;
"""
run_sql_df(sql)

## 12. Cómputo para herramientas de socios


In [ ]:
# 12.1 Approximate credit attribution by client app
sql = """
WITH client_hour_execution_cte AS (
  SELECT
    CASE
      WHEN client_application_id LIKE 'Go %'             THEN 'Go'
      WHEN client_application_id LIKE 'Snowflake UI %'   THEN 'Snowflake UI'
      WHEN client_application_id LIKE 'SnowSQL %'        THEN 'SnowSQL'
      WHEN client_application_id LIKE 'JDBC %'           THEN 'JDBC'
      WHEN client_application_id LIKE 'PythonConnector %' THEN 'Python'
      WHEN client_application_id LIKE 'ODBC %'           THEN 'ODBC'
      ELSE 'NOT YET MAPPED: ' || client_application_id
    END                                AS client_application_name,
    warehouse_name,
    DATE_TRUNC('hour', start_time)     AS start_time_hour,
    SUM(execution_time)                AS client_hour_execution_time
  FROM snowflake.account_usage.query_history qh
  JOIN snowflake.account_usage.sessions se
    ON se.session_id = qh.session_id
  WHERE warehouse_name IS NOT NULL
    AND execution_time > 0
    AND start_time > DATEADD(month, -1, CURRENT_TIMESTAMP())
  GROUP BY 1, 2, 3
),
hour_execution_cte AS (
  SELECT
    start_time_hour,
    warehouse_name,
    SUM(client_hour_execution_time) AS hour_execution_time
  FROM client_hour_execution_cte
  GROUP BY 1, 2
),
approximate_credits AS (
  SELECT
    A.client_application_name,
    C.warehouse_name,
    (A.client_hour_execution_time / NULLIF(B.hour_execution_time, 0)) * C.credits_used AS approximate_credits_used
  FROM client_hour_execution_cte A
  JOIN hour_execution_cte B
    ON A.start_time_hour = B.start_time_hour
   AND B.warehouse_name  = A.warehouse_name
  JOIN snowflake.account_usage.warehouse_metering_history C
    ON C.warehouse_name = A.warehouse_name
   AND C.start_time     = A.start_time_hour
)
SELECT
  client_application_name,
  warehouse_name,
  SUM(approximate_credits_used) AS approximate_credits_used
FROM approximate_credits
GROUP BY 1, 2
ORDER BY 3 DESC;
"""
run_sql_df(sql)

## 13. Cómputo para tablas híbridas


In [ ]:
# 13.1 Past year
sql = """
SELECT
  object_type,
  SUM(credits_used) AS total_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.HYBRID_TABLE_USAGE_HISTORY
GROUP BY 1;
"""
run_sql_df(sql)

In [ ]:
# 13.2 Past m days
sql = f"""
SELECT
  object_type,
  SUM(credits_used) AS total_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.HYBRID_TABLE_USAGE_HISTORY
WHERE start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
GROUP BY 1;
"""
run_sql_df(sql)

## 14. Cómputo para servicios Cortex


In [ ]:
# 14.1 Analyst
run_sql_df("SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_ANALYST_USAGE_HISTORY;")

In [ ]:
# 14.2 Fine-tuning
run_sql_df("SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FINE_TUNING_USAGE_HISTORY;")

In [ ]:
# 14.3 Functions
run_sql_df("SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY;")

In [ ]:
# 14.4 Functions by model
sql = f"SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY WHERE model_name = '{MODEL_NAME}';"
run_sql_df(sql)

In [ ]:
# 14.5 Functions query usage by query_id
sql = f"SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_QUERY_USAGE_HISTORY WHERE query_id = '{CORTEX_QUERY_ID}';"
run_sql_df(sql)

In [ ]:
# 14.6 Search (daily)
run_sql_df("SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_SEARCH_DAILY_USAGE_HISTORY;")

In [ ]:
# 14.7 Search serving
run_sql_df("SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_SEARCH_SERVING_USAGE_HISTORY;")

In [ ]:
# 14.8 Document AI
run_sql_df("SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.DOCUMENT_AI_USAGE_HISTORY;")

In [ ]:
# 14.9 Document AI per query (credits > 0.072)
run_sql_df("SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_DOCUMENT_PROCESSING_USAGE_HISTORY WHERE CREDITS_USED > 0.072;")

## 15. Cómputo para Snowflake Notebooks


In [ ]:
# 15.1 Hourly credit consumption by notebook
sql = f"""
SELECT *
FROM snowflake.account_usage.notebooks_container_runtime_history
WHERE notebook_name = '{NB_NAME}';
"""
run_sql_df(sql)

In [ ]:
# 15.2 Total cost to run a specific notebook
sql = f"""
SELECT
  notebook_name,
  SUM(credits) AS total_credits
FROM snowflake.account_usage.notebooks_container_runtime_history
WHERE notebook_name = '{NB_NAME}'
GROUP BY notebook_name;
"""
run_sql_df(sql)

In [ ]:
# 15.3 Total compute pool cost per notebook
sql = f"""
SELECT
  notebook_name,
  SUM(credits) AS total_credits
FROM snowflake.account_usage.notebooks_container_runtime_history
WHERE compute_pool_name = '{CP_NAME}'
GROUP BY notebook_name;
"""
run_sql_df(sql)

In [ ]:
# 15.4 Identify users who ran a specific notebook
sql = f"""
SELECT DISTINCT user_name
FROM snowflake.account_usage.notebooks_container_runtime_history
WHERE notebook_name = '{NB_NAME}';
"""
run_sql_df(sql)

---
### 16. Notas
- Valida los permisos y minimiza los rangos de tiempo para escaneos pesados.
- Considera agregar gráficos y alertas en un cuaderno de seguimiento para monitoreo continuo.
